# Evaluation of NN-Models and hybrid Algorithm for predicting SDAS

## Imports

In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip uninstall cellpose -y
!pip install cellpose

Found existing installation: cellpose 4.0.2
Uninstalling cellpose-4.0.2:
  Successfully uninstalled cellpose-4.0.2
  Using cached cellpose-4.0.2-py3-none-any.whl.metadata (21 kB)
Using cached cellpose-4.0.2-py3-none-any.whl (210 kB)

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from cellpose import core, utils, io, models, metrics



Welcome to CellposeSAM, cellpose v4.0.1! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [4]:
import importlib.metadata
print(importlib.metadata.version('cellpose'))

4.0.2


In [5]:
# === Standardbibliotheken ===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import time

# === Modell laden und evaluieren ===
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Metriken ===
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# für hybrides Verfahren

import sys
sys.path.append('../notebooks_HYBRID')  # Pfad zur Datei my_utils.py anpassen
from my_utils import (
    draw_contours_with_alpha,
    extract_line_segments,
    plot_line_segments_on_image,
    plot_line_midpoints_with_angles,
    group_line_segments,
    group_line_segments_with_KDTree,
    print_result_lines_over_img,
    calculate_avg_sdas,
    getResults,
    calculateMetrics
)

2025-05-12 14:12:24.420201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747059144.434773  977704 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747059144.439039  977704 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747059144.453010  977704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747059144.453028  977704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747059144.453030  977704 computation_placer.cc:177] computation placer alr

In [7]:
# Modelle importieren

cnn_model = load_model('../models/cnn_v1.keras')

ResNet50_model = load_model('../models/ResNet50_v2.keras', safe_mode=False)

EfficientNetB0_model = load_model('../models/EfficientNetB0_v2.keras')

paper_model = load_model('../models/paper_model.keras')

Cyto2_model = models.CellposeModel(gpu=True, model_type='cyto2')  #sometimes Cellpose(...) sometimes CellposeModel()

ValueError: The `{arg_name}` of this `Lambda` layer is a Python lambda. Deserializing it is unsafe. If you trust the source of the config artifact, you can override this error by passing `safe_mode=False` to `from_config()`, or calling `keras.config.enable_unsafe_deserialization().

## Parameter für hybrides Verfahren

In [ ]:
# PARAMETERS:

# Cellpose Parameters

diameter = 85      # is the smallest possible contour diameter

# Clustering Parameters

MIN_POINTS_PER_LINE = 5                                         # BDG-Norm: minimal number of elements in a line
MAX_POINTS_PER_LINE = 15                                        # maxmimum number of elements in a line
ANGLE_DIFF_THRESHOLD = 25                                       # maximum angle difference between current line and next line 
DISTANCE_THRESHOLD = 10                                         # maximum distance between contours of dendrites-structures
REGRESSION_DISTANCE_THRESHOLD = 100                             # maximum distance between current regression line and next line 
MIN_ANGLE_DIFF_REG_P_THRESHOLD = 75                             # minimal angle difference between current regression line and next line 
MICROMETER_PER_PIXEL = 0.728265817023213

## Evaluation auf allen Test Bildern

In [ ]:
test_dir = r'../data/bmw_test'

min_size = 200

results = []

# for all images in test_dir
for idx, img_name in enumerate(os.listdir(test_dir)):
    if img_name.endswith('.jpg') or img_name.endswith('.png'):
        
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        height, width = img.shape

        # HYBRIDES VERFAHREN:

        masks, flows, styles, imgs_dn = Cyto2_model.eval(img, diameter=diameter, channels=[0, 0])
        
        line_segments = extract_line_segments(masks)
        
        dendrite_clusters = group_line_segments_with_KDTree(
            line_segments, MAX_POINTS_PER_LINE, MAX_ANGLE_DIFF_REG_P_THRESHOLD,
            REGRESSION_DISTANCE_THRESHOLD, ANGLE_DIFF_THRESHOLD, DISTANCE_THRESHOLD,
            MIN_POINTS_PER_LINE, MICROMETER_PER_PIXEL
        )
        
        HYBRID_SDAS_pred = calculate_avg_sdas(dendrite_clusters)

        # NEURONALE NETZE (MIT TILES):
        
        # Calculate the number of tiles
        rows = height // min_size
        cols = width // min_size
        
        # Split the image into tiles
        images = []
        for i in range(rows):
            for j in range(cols):
                y_start = i * min_size
                x_start = j * min_size
                y_end = min(y_start + min_size, height)
                x_end = min(x_start + min_size, width)
                image = img[y_start:y_end, x_start:x_end]
                images.append(image)
        
        # Collect model predictions
        CNN_predictions = []
        ResNet50_predictions = []
        EfficientNetB0_predictions = []
        PaperModel_predictions = []
        
        for image in images:
            image = image.astype(np.float16) / 255.0  # normalize to [0, 1]
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))  # add channel dimension     
            
            CNN_prediction = CNN_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            CNN_predictions.append(CNN_prediction)
            
            ResNet50_prediction = ResNet50_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            ResNet50_predictions.append(ResNet50_prediction)
            
            EfficientNetB0_prediction = EfficientNetB0_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            EfficientNetB0_predictions.append(EfficientNetB0_prediction)

            PaperModel_prediction = paper_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            PaperModel_predictions.append(PaperModel_prediction)
        
        # Calculate the prediction
        CNN_S_value_pred = np.median(CNN_predictions) * 100 # * 100, because we normalized the data when we imported it 
        CNN_SDAS_pred = CNN_S_value_pred * F_value
        
        ResNet50_S_value_pred = np.median(ResNet50_predictions) * 100 
        ResNet50_SDAS_pred = ResNet50_S_value_pred * F_value
        
        EfficientNetB0_S_value_pred = np.median(PaperModel_predictions) * 100 
        PaperModel_SDAS_pred = PaperModel_S_value_pred * F_value

        Paper;pdel_S_value_pred = np.median(EfficientNetB0_predictions) * 100 
        EfficientNetB0_SDAS_pred = EfficientNetB0_S_value_pred * F_value

  
        # Extract the actual SDAS value from the filename
        try:
            SDAS_true = float(img_name.split('_')[1])
            
            CNN_std_pred = np.std(CNN_predictions) 
            CNN_error = abs(CNN_S_value_pred - SDAS_true)  
            
            ResNet50_std_pred = np.std(ResNet50_predictions) 
            ResNet50_error = abs(ResNet50_S_value_pred - SDAS_true)

            EfficientNetB0_std_pred = np.std(EfficientNetB0_predictions) 
            EfficientNetB0_error = abs(EfficientNetB0_S_value_pred - SDAS_true)  

            results.append((SDAS_true, HYBRID_SDAS_pred, CNN_SDAS_pred, CNN_std_pred, CNN_error, ResNet50_SDAS_pred, ResNet50_std_pred, ResNet50_error, EfficientNetB0_SDAS_pred, EfficientNetB0_std_pred, EfficientNetB0_error, PaperModel_SDAS_pred, PaperModel_std_pred, PaperModel_error))
        except:
            print(f"Warning: Couldn't extract SDAS-value from '{img_name}'!")

        if idx % 5 == 0:
            print(f"⏳ Processed {idx}/{len(os.listdir(test_dir))} images...")

In [ ]:
# Hilfsfunktion zur MAPE-Berechnung (mit 0-Division-Schutz)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-8, None))) * 100

In [ ]:
# Entpacke Werte aus den Resultaten
y_true = [row[0] for row in results]

y_pred_HYBRID = [row[1] for row in results]
y_pred_CNN = [row[2] for row in results]
y_pred_ResNet50 = [row[5] for row in results]
y_pred_EfficientNetB0 = [row[8] for row in results]
y_pred_PaperModel = [row[11] for row in results]

### Metriken

In [ ]:
# Alle Modelle und ihre Vorhersagen
models = {
    "HYBRID": y_pred_HYBRID,
    "CNN": y_pred_CNN,
    "ResNet50": y_pred_ResNet50,
    "EfficientNetB0": y_pred_EfficientNetB0
    "Paper Model" : y_pred_PaperModel
}

# Ausgabe der Metriken
for name, y_pred in models.items():
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n🔍 {name} Modell:")
    print(f"   MSE:   {mse:.4f}")
    print(f"   RMSE:  {rmse:.4f}")
    print(f"   MAE:   {mae:.4f}")
    print(f"   MAPE:  {mape:.2f}%")
    print(f"   R²:    {r2:.4f}")

### Abweichung Pred von True SDAS

In [ ]:
# Konvertiere ground truth zu Array (falls noch nicht geschehen)
y_true_array = np.array([row[0] for row in results])

# Vorhersagen extrahieren
preds = {
    "HYBRID": np.array([row[1] for row in results]),
    "CNN": np.array([row[2] for row in results]),
    "ResNet50": np.array([row[5] for row in results]),
    "EfficientNetB0": np.array([row[8] for row in results]),
    "Paper Model": np.array([row[11] for row in results]),
}

# Plot erstellen
plt.figure(figsize=(12, 12))
for i, (model_name, y_pred_array) in enumerate(preds.items(), start=1):
    plt.subplot(2, 2, i)
    plt.plot([0, 100], [0, 100], 'r--', label='Ideal')  # Diagonal
    plt.scatter(y_true_array, y_pred_array, alpha=0.7)
    plt.xlabel("Ground Truth SDAS")
    plt.ylabel("Predicted SDAS")
    plt.title(f"{model_name} Model")
    plt.axis("equal")
    plt.legend()

plt.tight_layout()
plt.suptitle("Predicted vs. Ground Truth SDAS for All Models", fontsize=16, y=1.02)
plt.show()

In [ ]:
# Berechne Pearson Korrelation für jedes Modell
for model_name, y_pred_array in preds.items():
    corr, _ = pearsonr(y_true_array, y_pred_array)
    print(f"Pearson correlation for {model_name}: {corr:.4f}")

### Zusammenhang von Varianz und Error

In [ ]:
# Daten extrahieren
model_data = {
    "CNN": {
        "stds": [row[3] for row in results],
        "errors": [row[4] for row in results],
    },
    "ResNet50": {
        "stds": [row[6] for row in results],
        "errors": [row[7] for row in results],
    },
    "EfficientNetB0": {
        "stds": [row[9] for row in results],
        "errors": [row[10] for row in results],
    }

    "Paper Model": {
        "stds": [row[12] for row in results],
        "errors": [row[13] for row in results],
    }
}

# Plot erstellen
plt.figure(figsize=(18, 5))
for i, (model_name, data) in enumerate(model_data.items(), start=1):
    plt.subplot(1, 3, i)
    plt.scatter(data["stds"], data["errors"], alpha=0.7, edgecolor='k')
    plt.xlabel("Standard Deviation of Tile Predictions")
    plt.ylabel("Prediction Error")
    plt.title(f"{model_name}: Std vs. Error")
    plt.grid(True)

plt.tight_layout()
plt.suptitle("Prediction Variance vs. Prediction Error", fontsize=16, y=1.05)
plt.show()

## Verarbeitungszeit

In [ ]:
# Die Anzahl der Bilder
num_images = len([img_name for img_name in os.listdir(test_dir) if img_name.endswith('.jpg') or img_name.endswith('.png')])

# Timer für jedes Verfahren initialisieren
hybrid_time = 0
cnn_time = 0
resnet50_time = 0
efficientnetb0_time = 0
paperModel_time = 0

# for all images in test_dir
for idx, img_name in enumerate(os.listdir(test_dir)):
    if img_name.endswith('.jpg') or img_name.endswith('.png'):
        
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        height, width = img.shape

        # HYBRIDES VERFAHREN:
        hybrid_start_time = time.time()  # Startzeit für das hybride Verfahren
        masks, flows, styles, imgs_dn = Cyto2_model.eval(img, diameter=diameter, channels=[0, 0])
        line_segments = extract_line_segments(masks)
        dendrite_clusters = group_line_segments_with_KDTree(
            line_segments, MAX_POINTS_PER_LINE, MAX_ANGLE_DIFF_REG_P_THRESHOLD,
            REGRESSION_DISTANCE_THRESHOLD, ANGLE_DIFF_THRESHOLD, DISTANCE_THRESHOLD,
            MIN_POINTS_PER_LINE, MICROMETER_PER_PIXEL
        )
        HYBRID_SDAS_pred = calculate_avg_sdas(dendrite_clusters)
        hybrid_time += time.time() - hybrid_start_time  # Addiere Zeit für das hybride Verfahren

        # NEURONALE NETZE (MIT TILES):
        
        cnn_start_time = time.time()  # Startzeit für das CNN
        CNN_predictions = []
        for image in images:
            image = image.astype(np.float16) / 255.0  # normalize to [0, 1]
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))  # add channel dimension     
            CNN_prediction = CNN_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            CNN_predictions.append(CNN_prediction)
        cnn_time += time.time() - cnn_start_time  # Addiere Zeit für das CNN

        resnet50_start_time = time.time()  # Startzeit für ResNet50
        ResNet50_predictions = []
        for image in images:
            image = image.astype(np.float16) / 255.0
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))
            ResNet50_prediction = ResNet50_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            ResNet50_predictions.append(ResNet50_prediction)
        resnet50_time += time.time() - resnet50_start_time  # Addiere Zeit für ResNet50

        efficientnetb0_start_time = time.time()  # Startzeit für EfficientNetB0
        EfficientNetB0_predictions = []
        for image in images:
            image = image.astype(np.float16) / 255.0
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))
            EfficientNetB0_prediction = EfficientNetB0_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            EfficientNetB0_predictions.append(EfficientNetB0_prediction)
        efficientnetb0_time += time.time() - efficientnetb0_start_time  # Addiere Zeit für EfficientNetB0

        paperModel_time_start_time = time.time()  # Startzeit für EfficientNetB0
        paperModel_predictions = []
        for image in images:
            image = image.astype(np.float16) / 255.0
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))
            paperModel_prediction = paper_model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            paperModel_predictions.append(paperModel_prediction)
        paperModel_time += time.time() - paperModel_time_start_time  # Addiere Zeit für EfficientNetB0

        if idx % 5 == 0:
            print(f"⏳ Processed {idx}/{len(os.listdir(test_dir))} images...")

# Berechnung der durchschnittlichen Verarbeitungszeit pro Bild
hybrid_avg_time = hybrid_time / num_images
cnn_avg_time = cnn_time / num_images
resnet50_avg_time = resnet50_time / num_images
efficientnetb0_avg_time = efficientnetb0_time / num_images
paperModel_avg_time = paperModel_time / num_images

print(f"Durchschnittliche Verarbeitungszeit pro Bild:")
print(f"HYBRID-Verfahren: {hybrid_avg_time:.4f} Sekunden")
print(f"CNN-Modell: {cnn_avg_time:.4f} Sekunden")
print(f"ResNet50-Modell: {resnet50_avg_time:.4f} Sekunden")
print(f"EfficientNetB0-Modell: {efficientnetb0_avg_time:.4f} Sekunden")
print(f"Paper-Modell: {paperModel_avg_time:.4f} Sekunden")

## Test auf einzelnen Bildern

In [ ]:
img_name = 'AC_39_B9.jpg'  # Replace with the image you want to test

In [ ]:
def process_image(test_dir, img_name, model):
    img_path = os.path.join(test_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape
    
    # Calculate the number of tiles
    rows = height // min_size
    cols = width // min_size
    
    # Split the image into tiles
    images = []
    predictions = []
    for i in range(rows):
        for j in range(cols):
            y_start = i * min_size
            x_start = j * min_size
            y_end = min(y_start + min_size, height)
            x_end = min(x_start + min_size, width)
            image = img[y_start:y_end, x_start:x_end]
            images.append(image)
            
            # Normalize and resize the image for prediction
            image = image.astype(np.float16) / 255.0  # normalize to [0, 1]
            image_resized = tf.image.resize(image[..., np.newaxis], (200, 200))  # Add channel dimension            
            prediction = model.predict(np.expand_dims(image_resized, axis=0), verbose=0)[0][0]
            predictions.append(prediction * 100 * F_value)
    
    return img, predictions, rows, cols

def plot_heatmap_with_image(img, predictions, rows, cols, min_size):
    # Reshape predictions to match the number of tiles
    heatmap = np.array(predictions).reshape((rows, cols))
    
    # Create a figure
    plt.figure(figsize=(10, 10))
    
    # Display the original image
    plt.imshow(img, cmap='gray', extent=[0, img.shape[1], img.shape[0], 0])
    
    # Overlay the heatmap on top of the image
    plt.imshow(heatmap, cmap='jet', alpha=0.5, extent=[0, img.shape[1], img.shape[0], 0])
    
    # Add color bar (legend for the heatmap)
    cbar = plt.colorbar()
    cbar.set_label('SDAS Prediction')

    # Show the plot
    plt.title("Image with SDAS Heatmap Overlay")
    plt.show()

In [ ]:
# CNN Model

img, predictions, rows, cols = process_image(test_dir, img_name, CNN_model)

print(np.median(predictions))

plot_heatmap_with_image(img, predictions, rows, cols, min_size)

In [ ]:
# ResNet50 model

img, predictions, rows, cols = process_image(test_dir, img_name, ResNet50_model)

print(np.median(predictions))

plot_heatmap_with_image(img, predictions, rows, cols, min_size)

In [ ]:
# EfficientNetB0 model

img, predictions, rows, cols = process_image(test_dir, img_name, EfficientNetB0_model)

print(np.median(predictions))

plot_heatmap_with_image(img, predictions, rows, cols, min_size)

In [ ]:
# CNN with paper weights

img, predictions, rows, cols = process_image(test_dir, img_name, paper_model)

print(np.median(predictions))

plot_heatmap_with_image(img, predictions, rows, cols, min_size)

In [ ]:
# HYBRID model

img = cv2.imread(image_path)

masks, flows, styles, imgs_dn = model.eval(img, diameter=diameter, channels= [0,0]) # Diameter ist entscheidend für sinnvolle Umrandunungen! -> kleinst möglicher Durchmesser der füllenden Linie

img_contours = draw_contours_with_alpha(img, masks, alpha=0.75)

In [ ]:
line_segments = extract_line_segments(masks)
img_lines = plot_line_segments_on_image(img, line_segments)

In [ ]:
plot_line_midpoints_with_angles(line_segments, img_lines)

In [ ]:
lines = group_line_segments_with_KDTree(line_segments, MAX_POINTS_PER_LINE, MIN_ANGLE_DIFF_REG_P_THRESHOLD,
                         REGRESSION_DISTANCE_THRESHOLD, ANGLE_DIFF_THRESHOLD, DISTANCE_THRESHOLD,
                         MIN_POINTS_PER_LINE, MICROMETER_PER_PIXEL)

In [ ]:
print_result_lines_over_img(lines, img)

In [ ]:
print_result_lines_over_img(lines, img_lines)

In [ ]:
print_result_lines_over_img(lines, img_contours)